In [1]:
from keras import layers
from keras import Input
from keras.models import Model

In [3]:
# A neural net that takes as input social media posts and predicts multiple data like gender, age, income

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# Give labels to the output layers as the name property
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
        [age_prediction, income_prediction, gender_prediction])

In [ ]:
# Multiple output layers require multiple loss functions
# The simplest way to treat multiple loss functions is to sum the value

# model.compile(optimizer='rmsprop',
#     loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# Equivalent to above (better for not risking messing up the order)
model.compile(optimizer='rmsprop', loss={'age': 'mse',
    'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'})

In [5]:
# Weights can be given to the losses to balance them out.
# Because MSE returns values typically within 3~5, while
# binary cross-entropy can return values as low as 0.1

model.compile(optimizer='rmsprop', loss={'age': 'mse',
    'income': 'categorical_crossentropy',
    'gender': 'binary_crossentropy'}, loss_weights={'age': 0.25,
    'income': 1., 'gender': 10.})


In [8]:
# Values for training can be passed as a dictionary as well

model.fit(posts, {'age': age_targets,
                          'income': income_targets,
                          'gender': gender_targets},
                  epochs=10, batch_size=64)

NameError: name 'posts' is not defined